---

## Data Analysis

- This file differs from [2_data_analysis_1_base_data.ipynb](2_data_analysis_1_base_data.ipynb) in that it:
    - scales the base cleaned data created in [1_data_cleaning.ipynb](1_data_cleaning.ipynb).

Source dataset: 247076 rows × 37 columns
Processed and analyzed dataset: 247076 rows × 37 columns

---

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys
import ml_clean_feature as mlclean

import warnings
warnings.filterwarnings("ignore")

In [2]:
start_time = time.time()

---

## 1. Read the cleaned dataset from file

---

In [3]:
year                        = config.year

clean_file                  = config.clean_file
optimization_report         = config.optimization_report

# report_path                 = config.report_path
# file_label                  = dataset_label.lower().replace(' ','_')
# detailed_performance_report = report_path + file_label + '_detailed_performance_report.txt'

print(f"Year:                        {year}")
print(f"Clean File:                  {clean_file}")
# print(f"Performance Report:          {performance_report}")
# print(f"Detailed Performance Report: {detailed_performance_report}")

Year:                        2021
Clean File:                  data/brfss_2021_clean.parquet.gzip


In [4]:
# Read final cleaned dataset from parquet file
df = pd.read_parquet(clean_file, engine="fastparquet")

In [5]:
diabetes_labels = df.columns

In [6]:
df.shape

(247076, 37)

---

## 2. Prepare the dataset for analysis

- Split the dataset into features and labels.
- Split the dataset into training and testing sets.
- Scale the dataset

---

In [7]:
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

target = 'diabetes'
# Dictionary defining modification to be made to the base dataset
operation_dict = {  'target_column'     :  target,
                    'convert_to_binary' :  True,
                    'scaler'            : 'standard', # options: none, standard, minmax
                    'random_sample'     : 'undersample'      # options: none, undersample, oversample
                    }

if operation_dict:
    dataset_label = 'binary_' + operation_dict['scaler'] + '_' + operation_dict['random_sample']
else:
    dataset_label = operation_dict['scaler'] + '_' + operation_dict['random_sample']

# This insures that df if not modified during the call to modify_base_dataset()
df_modified = df.copy()

# Modify the base dataset
# data is returned where: X_train, X_test, y_train, y_test = data
data = mlanlys.modify_base_dataset(df_modified, operation_dict)

Base Dataset Modifications in Process
-------------------------------------
**Operation:target_column  diabetes
**Operation:convert_to_binary  True
  -- Converting dataset to binary (0,1) from (0,1,2)


****Cleaning Feature: diabetes
  Initial Unique features in [diabetes]:  [0. 1. 2.]
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: ********* NO Parameters were specified *********
  FINAL Unique features in [diabetes]:  [0. 1.]
**Operation:scaler  standard
  -- Performing train_test_split on dataframe with target:'diabetes'
     -- Run automatically before scalar or random_sample operations
  -- Performing StandardScaler on X_train: Updates X_train, y_test
**Operation:random_sample  undersample
  -- Performing RandomUnderSampler on X_train, y_train: Updates X_train, y_train

Dataframe, Train Test Summary
-----------------------------
Dataframe: (247076, 37)  Data:4, X_train:49468, y_train:49468, X_test:61769, y_test:61769
ValueCount

In [9]:
# Print some statistics about the original df and the modified dataframe
print(f"Original Dataframe")
print(f"------------------")
print(f"df.shape: {df.shape}")
print(f"df[{target}].value_counts:  {df[target].value_counts()}")

print(f"\nModified Dataframe")
print(f"------------------")
print(f"df_modified.shape: {df_modified.shape}")
print(f"df_modified[{target}].value_counts:  {df_modified[target].value_counts()}")

Original Dataframe
------------------
df.shape: (247076, 37)
df[diabetes].value_counts:  diabetes
0.0    208389
2.0     33033
1.0      5654
Name: count, dtype: int64

Modified Dataframe
------------------
df_modified.shape: (247076, 37)
df_modified[diabetes].value_counts:  diabetes
0.0    214043
1.0     33033
Name: count, dtype: int64


In [10]:
X_train, X_test, y_train, y_test = data
print(f"Dataframe: {df_modified.shape}  Data:{len(data)}, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")

Dataframe: (247076, 37)  Data:4, X_train:49468, y_train:49468, X_test:61769, y_test:61769


In [11]:
y_train.value_counts()

diabetes
0.0    24734
1.0    24734
Name: count, dtype: int64

In [12]:
y_test.value_counts()

diabetes
0.0    53470
1.0     8299
Name: count, dtype: int64

---

## 3. Optimization Prep

---

#### 3.1 Pre-optimization metric results

- The summary report of the metrics for all pre-optimization runs is here:  [performance_report.txt](reports/performance_report.txt)
- The details of the runs are contained in these file:
    - [base_dataset_detailed_performance_report.txt](reports/base_dataset_detailed_performance_report.txt)
    - [binary_dataset_detailed_performance_report.txt](reports/binary_dataset_detailed_performance_report.txt)
    - [minmax_scaled_dataset_detailed_performance_report.txt](reports/minmax_scaled_dataset_detailed_performance_report.txt)
    - [randomoversample_dataset_detailed_performance_report.txt](reports/randomoversample_dataset_detailed_performance.txt)
    - [cluster_dataset_detailed_performance_report.txt](reports/cluster_dataset_detailed_performance_report.txt)
    - [randomundersampled_dataset_detailed_performance_report.txt](reports/randomundersampled_dataset_detailed_performance_report.txt)
    - [minmax_scaled_dataset_detailed_performance_report.txt](reports/minmax_scaled_dataset_detailed_performance_report.txt)
    - [smoteen_dataset_detailed_performance_report.txt](reports/smoteen_dataset_detailed_performance_report.txt)
    - [standard_scaled_dataset_detailed_performance_report.txt](reports/standard_scaled_dataset_detailed_performance_report.txt)
    - [smote_dataset_detailed_performance_report.txt](reports/smote_dataset_detailed_performance_report.txt)

#### 3.2 Optimization Dataset used

**Note:**  Modify the dataset as desired in Section 2.
<br><br>
Currently the dataset uses the Base dataset for 2015 with the following modifications:
- **Target converted to Binary**:  (0,1) from (0,1,2).
    - Base:  0: No diabetes, 1: Pre-diabetes, 2: have diabetes
    - binary: 0: No diabetes/pre-diabetes, 1: have diabetes
- Scaled the data with **StandardScaler**
- Resampled the data with **RandomUnderSampler**

#### 3.3 Notes:  Hyperparameter Spaces

| Section| Classifier | hyperparameter space |
| -------| ---------- | -------------------- |
|4.1| **DecisionTreeClassifier:** | dt_param_distributions |
|4.2| **LogisticRegression:** | lr_param_distributions |
|4.3| **AdaBoostClassifier:** | ada_param_distributions |
|4.4| **GradientBoostingClassifier:** | gb_param_distributions |
|4.5| **RandomForestClassifier:** | rf_param_distributions |
|4.6| **ExtraTreesClassifier:** | et_param_distributions |
|4.7| **KNeighborsClassifier:** | knn_param_distributions |


In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ParameterSampler
from scipy.stats import uniform

In [14]:
# Define the hyperparameter search space for the Decision Tree Classifier:


dt_param_distributions = {
    'max_depth': [int(x) for x in uniform(1, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
}

# Define the hyperparameter search space for the AdaBoostClassifier

ada_param_distributions = {
#    'alogrithm': ['SAMME'],
    'n_estimators': [int(x) for x in uniform(10, 1000).rvs(10)],
    'learning_rate': [x for x in uniform(0.001, 1).rvs(10)],
    'estimator': [DecisionTreeClassifier(**dt_params) for dt_params in
                       ParameterSampler(dt_param_distributions, n_iter=10)]
}

# GradientBoostingClassifier:
gb_param_distributions = {
    'n_estimators': [int(x) for x in uniform(100, 1000).rvs(10)],
    'max_depth': [int(x) for x in uniform(2, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
    'max_features': ['sqrt', 'log2', None],
    'learning_rate': [x for x in uniform(0.01, 0.5).rvs(10)],
    'subsample': [x for x in uniform(0.5, 1).rvs(10)],
}

# RandomForestClassifier
rf_param_distributions = {
    'n_estimators': [int(x) for x in uniform(100, 1000).rvs(10)],
    'max_depth': [int(x) for x in uniform(2, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
}

# KNeighborsClassifier
knn_param_distributions = {
    'n_neighbors': [int(x) for x in uniform(1, 20).rvs(10)],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'leaf_size': [int(x) for x in uniform(10, 60).rvs(10)],
}

# LogisticRegression
lr_param_distributions = {
    'C': [x for x in uniform(0.001, 10).rvs(10)],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [int(x) for x in uniform(100, 1000).rvs(10)],
}

# ExtraTreesClassifier 
et_param_distributions = {
    'n_estimators': [int(x) for x in uniform(100, 1000).rvs(10)],
    'max_depth': [int(x) for x in uniform(2, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'max_leaf_nodes': [int(x) for x in uniform(10, 100).rvs(10)],
    'min_impurity_decrease': [x for x in uniform(0, 0.1).rvs(10)],
}

## 4. Optimization Analysis

In [15]:
# Note:  Modify the dataset as desired in Section 2.
# Currently the dataset uses the Base dataset for 2015 with the following modifications

In [16]:
# Dataset summary:
X_train, X_test, y_train, y_test = data
print(f"Dataset Lens, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")
print(f"y_train value_counts: {y_train.value_counts()}")
print(f"y_test value_counts: {y_test.value_counts()}")


Dataset Lens, X_train:49468, y_train:49468, X_test:61769, y_test:61769
y_train value_counts: diabetes
0.0    24734
1.0    24734
Name: count, dtype: int64
y_test value_counts: diabetes
0.0    53470
1.0     8299
Name: count, dtype: int64


In [17]:
dataset_label

'binary_standard_undersample'

#### 4.1 Perform Optimization:  DecisionTreeClassifier

- **model:** DecisionTreeClassifier 
- **Optimization:** RandomizedSearchCV


In [18]:
# reload any changes to mlanlys
importlib.reload(mlanlys)
from sklearn.tree import DecisionTreeClassifier

# Model
model = DecisionTreeClassifier()

# Define the parameter grid
param_distributions = {
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)],  # Depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider for the best split
}

optimize_report_params = {
    'param_distributions': param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 1,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: Model: DecisionTreeClassifier: Parameters: {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10}
------------------------------------------------------------------------
---------- optimizeding Data Performance
------------------------------------------------------------------------
Confusion Matrix
[[36466 17004]
 [ 1772  6527]]

-----------------------
optimized score: 0.696
Balanced Accuracy Score: 0.7342
ROC AUC Score: 0.7993
Mean Squared Error: 0.304
------------------------------
--- Classification values
------------------------------
Accuracy: 0.696
Precision: 0.2774
Recall: 0.7865
F1-score: 0.4101
Specificity: 0.682
False Positive Rate: 0.318
Matthews Correlation Coefficient: 0.329

-----------------------
Classification Report
              precision    recall  f1-score   support

         0.0       0.95      0.68      0.80     53470
         1.0       0.28     

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient,Optimizer,best_parameters
0,binary_standard_undersample,DecisionTreeClassifier,un-optimized,0.6683,0.6661,0.6661,0.3317,0.6683,0.2372,0.6630,0.3494,0.6692,0.3308,0.2339,RandomizedSearchCV,None
1,binary_standard_undersample,DecisionTreeClassifier,optimized,0.6960,0.7342,0.7993,0.3040,0.6960,0.2774,0.7865,0.4101,0.6820,0.3180,0.3290,RandomizedSearchCV,"{'min_samples_split': 10, 'min_samples_leaf': ..."


#### 4.2 Perform Optimization:  LogisticRegression

- **model:** LogisticRegression 
- **Optimization:** RandomizedSearchCV

In [20]:
# reload any changes to mlanlys
importlib.reload(mlanlys)
from sklearn.linear_model import LogisticRegression


# Model
model = LogisticRegression(max_iter=5000)

# Define the parameter grid
param_distributions = {
    'C': uniform(loc=0, scale=4),   # Regularization parameter
    'penalty': ['l1', 'l2'],        # Regularization type
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']  # Solvers that support L2 regularization
}


optimize_report_params = {
    'param_distributions': param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 3,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END C=1.49816047538945, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END C=1.49816047538945, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END C=1.49816047538945, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END C=1.49816047538945, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END C=1.49816047538945, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=2.9279757672456204, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END C=2.9279757672456204, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5] END C=2.9279757672456204, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 4/5] END C=2.9279757672456204, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 5/5] END C=2.9279757672456204, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END C=0.6240745617

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient,Optimizer,best_parameters
0,binary_standard_undersample,LogisticRegression,un-optimized,0.7285,0.7469,0.824,0.2715,0.7285,0.3010,0.7721,0.4331,0.7218,0.2782,0.3544,RandomizedSearchCV,None
1,binary_standard_undersample,LogisticRegression,optimized,0.7285,0.7471,0.824,0.2715,0.7285,0.3011,0.7726,0.4333,0.7216,0.2784,0.3546,RandomizedSearchCV,"{'C': 0.2059150049999574, 'penalty': 'l1', 'so..."


#### 4.3 Perform Optimization: AdaBoostClassifier

- **model:** AdaBoostClassifier 
- **Optimization:** RandomizedSearchCV

In [20]:
# reload any changes to mlanlys
importlib.reload(mlanlys)
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import uniform

# Model
model = AdaBoostClassifier(random_state=1)

# Define the parameter grid
ada_param_distributions = {
#    'alogrithm': ['SAMME'],
    'n_estimators': [int(x) for x in uniform(10, 1000).rvs(10)],
    'learning_rate': [x for x in uniform(0.001, 1).rvs(10)],
    'estimator': [DecisionTreeClassifier(**dt_params) for dt_params in
                       ParameterSampler(dt_param_distributions, n_iter=10)]
}
# # valid estimators: ['algorithm', 'estimator', 'learning_rate', 'n_estimators', 'random_state']


optimize_report_params = {
    'param_distributions': ada_param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 0,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report

/home/jeffco/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/jeffco/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/jeffco/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/jeffco/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the S

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,binary_standard_undersample,AdaBoostClassifier,un-optimized,0.7369,0.7555,0.8341,0.2631,0.7369,0.3115,0.7811,0.4454,0.7300,0.2700,0.3692
1,binary_standard_undersample,AdaBoostClassifier,optimized,0.7303,0.7596,0.8371,0.2697,0.7303,0.3084,0.7997,0.4451,0.7195,0.2805,0.3721


#### 4.4 Perform Optimization: GradientBoostingClassifier

- **model:** GradientBoostingClassifier 
- **Optimization:** RandomizedSearchCV

In [ ]:
# GradientBoostingClassifier:

# Model
model = GradientBoostingClassifier(random_state=1)

gb_param_distributions = {
    'n_estimators': [int(x) for x in uniform(100, 1000).rvs(10)],
    'max_depth': [int(x) for x in uniform(2, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
    'max_features': ['sqrt', 'log2', None],
    'learning_rate': [x for x in uniform(0.01, 0.5).rvs(10)],
    'subsample': [x for x in uniform(0.5, 1).rvs(10)]
}

optimize_report_params = {
    'param_distributions': gb_param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 0,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report

#### 4.5 Perform Optimization: RandomForestClassifier

- **model:** RandomForestClassifier 
- **Optimization:** RandomizedSearchCV

In [22]:
# RandomForestClassifier

# Model
model = RandomForestClassifier()

rf_param_distributions = {
    'n_estimators': [int(x) for x in uniform(100, 1000).rvs(10)],
    'max_depth': [int(x) for x in uniform(2, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
}

optimize_report_params = {
    'param_distributions': rf_param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 0,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report


#### 4.6 Perform Optimization: ExtraTreesClassifier

- **model:** ExtraTreesClassifier 
- **Optimization:** RandomizedSearchCV

In [ ]:
# ExtraTreesClassifier 

# Model
model = ExtraTreesClassifier(random_state=1)

et_param_distributions = {
    'n_estimators': [int(x) for x in uniform(100, 1000).rvs(10)],
    'max_depth': [int(x) for x in uniform(2, 10).rvs(10)],
    'min_samples_split': [int(x) for x in uniform(2, 20).rvs(10)],
    'min_samples_leaf': [int(x) for x in uniform(1, 10).rvs(10)],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'max_leaf_nodes': [int(x) for x in uniform(10, 100).rvs(10)],
    'min_impurity_decrease': [x for x in uniform(0, 0.1).rvs(10)],
}

optimize_report_params = {
    'param_distributions': et_param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 0,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report

#### 4.7 Perform Optimization: KNeighborsClassifier

- **model:** KNeighborsClassifier 
- **Optimization:** RandomizedSearchCV

**NOte:** KNN took a long time to do a single run.  So it might need to be skipped for optimization.

In [ ]:
# KNeighborsClassifier

# Model
k_value = 3
model = KNeighborsClassifier(n_neighbors=k_value)

knn_param_distributions = {
    'n_neighbors': [int(x) for x in uniform(1, 20).rvs(10)],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'leaf_size': [int(x) for x in uniform(10, 60).rvs(10)],
}

optimize_report_params = {
    'param_distributions': knn_param_distributions,
    'optimize_path': config.optimize_path,
    'dataset': dataset_label,
    'optimization_report': config.optimization_report,
    'scoring': 'accuracy',
    'verbose': 0,
    'print_results': False
    }

optimize_report = mlanlys.run_random_search(model, optimize_report_params, data)

optimize_report

---
---
---
---

In [ ]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

---

## 4. Conclusions

- The confusion matrix shows more TP/TN values and Fewer FP/FN values 
- Classification accuracy increased from .66 to .68
- Balanced accuracy increased from .66 to .72
- ROC AUC Score increased from .66 to .78

**Summary:  Decision Tree Classifiering Performance Improved**

---

In [ ]:


#
